# Introduction: Sorting out cities and city districts

Because on the website, for real estate listings, user validation was not set in place for verifying city and city district information, in some of the real estete entries there is wrong information about the location.

After initial data cleaning, in this jupyter notebook, we will try to sort out and set the correct information for the location of a real estete entry.

# 1. Connecting to the database

In [1]:
# Importting libraries
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading db connection parameters from json file
db_params = None
with open('../database/db_connetion.json') as db_file:
    # Loading db connection parameters 
    db_params = json.load(db_file)

In [3]:
# Connecting to the database
conn = psycopg2.connect(
    host=db_params['host'], 
    port=db_params['port'], 
    database=db_params['database'], 
    user=db_params['user'], 
    password=db_params['password']
    )

In [4]:
# Getting the real estate data from database
real_estate = pd.read_sql_query('Select * from properties_clean', conn)

# 2. Removing serbian specific latin characters

For convenience and data consistency, first we will remove any specific serbian latin characters.

In [5]:
def _replace_latin_characters(value):
    if value is None:
        return None
    value = value.lower()
    value = value.replace('ć', 'c')
    value = value.replace('č', 'c')
    value = value.replace('š', 's')
    value = value.replace('đ', 'dj')
    value = value.replace('ž', 'z')
    return value

In [6]:
print(f'Changing latin characters for f{len(real_estate)} real estate entries')
for index in range(0, len(real_estate)):
    current_real_estate = real_estate.iloc[index, :]
    real_estate.at[index, 'location_city'] = _replace_latin_characters(current_real_estate['location_city'])
    real_estate.at[index, 'location_city_district'] = _replace_latin_characters(current_real_estate['location_city_district']) 

Changing latin characters for f21604 real estate entries


# 3. Finding wrong entries for city and city distric location

Using geographical knowlegde of Serbia and it's couple of biggest cities, we should be able to detect where there are a wrong information about real estate locations.

In [7]:
# Listing top cities by number of listed properties withing them
real_estate.groupby(['location_city']).count().sort_values('id', ascending=False).head(50)

,id,listing_type,property_type,price,location_city_district,area_property,area_land,construction_type,num_floors_building,apartment_floor,registered,heating_type,num_rooms,num_bathrooms,source
location_city,,,,,,,,,,,,,,,
beograd,11487,11487,11487,11487,11462,11487,1454,2359,4857,8013,11487,10465,8999,11487,11487
novi sad,4221,4221,4221,4221,4177,4221,320,1714,2406,3314,4221,3265,3474,4221,4221
gradske lokacije,899,899,899,899,899,899,129,142,244,554,899,620,592,899,899
kragujevac,687,687,687,687,480,687,351,103,177,253,687,470,263,687,687
nis,523,523,523,523,453,523,58,65,100,199,523,282,224,523,523
srbija,523,523,523,523,523,523,321,182,61,83,523,391,89,523,523
stari grad,228,228,228,228,228,228,11,42,135,202,228,204,207,228,228
subotica,125,125,125,125,125,125,96,32,8,14,125,88,15,125,125
palilula,119,119,119,119,119,119,61,52,32,38,119,90,42,119,119


### Now we know that for some entries, city district information is misplaced with city information. In city filed, we can actually find infomation of city district:
- gradske lokacije	
- medijana
- srbija
- stari grad	
- palilula
- vozdovac
- zvezdara
- zemun
- novi beograd	
- ...

We will try to set the right city and city district information on them. 

# 4. Setting the right location information 

## 4.1. Properties with "gradske lokacije" city value

In [8]:
# All entries with "Gradske lokacije" city info
indexes_to_replace = real_estate[real_estate['location_city'] == 'gradske lokacije'].index
real_estate.iloc[indexes_to_replace, :]

,id,listing_type,property_type,price,location_city,location_city_district,area_property,area_land,construction_type,num_floors_building,apartment_floor,registered,heating_type,num_rooms,num_bathrooms,source
44,113,s,a,62000.0,gradske lokacije,medijana,72.0,NaN,None,NaN,3.0,True,Centralno,3.0,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
45,114,s,a,82000.0,gradske lokacije,medijana,80.0,NaN,None,NaN,0.0,True,Struja,4.0,2.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
46,115,s,a,85000.0,gradske lokacije,medijana,96.0,NaN,None,NaN,7.0,True,Centralno,3.5,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
133,271,s,a,48000.0,gradske lokacije,medijana,47.0,NaN,None,7.0,7.0,True,Centralno,1.0,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
137,275,s,a,41000.0,gradske lokacije,medijana,42.0,NaN,2020,3.0,3.0,True,Struja,1.5,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20728,21523,r,a,300.0,gradske lokacije,medijana,50.0,NaN,None,NaN,1.0,False,Centralno,2.0,1.0,https://www.4zida.rs/izdavanje/stanovi/nis/ogl...
20923,21718,r,a,300.0,gradske lokacije,medijana,50.0,NaN,None,NaN,1.0,False,Etažno,1.5,1.0,https://www.4zida.rs/izdavanje/stanovi/nis/ogl...
20973,21768,r,a,270.0,gradske lokacije,medijana,55.0,NaN,None,NaN,0.0,True,Centralno,2.5,1.0,https://www.4zida.rs/izdavanje/stanovi/nis/ogl...
20985,21780,r,a,220.0,gradske lokacije,pantelej,44.0,NaN,None,NaN,1.0,False,None,1.5,1.0,https://www.4zida.rs/izdavanje/stanovi/nis/ogl...


In [9]:
# Trying to get the right city information from source link
for index in indexes_to_replace:
    print(real_estate.iloc[index,-1])

https://www.4zida.rs/prodaja/stanovi/nis/oglas/cair/5f0452680c7cde014c6fa13b
https://www.4zida.rs/prodaja/stanovi/nis/oglas/centar/5e6f584b0c7cde72d026ec5d
https://www.4zida.rs/prodaja/stanovi/nis/oglas/centar/5e07272a0c7cde4e447c7b82
https://www.4zida.rs/prodaja/stanovi/nis/oglas/radoja-dakica/5f12a6569a30911ac4100d08
https://www.4zida.rs/prodaja/stanovi/nis/oglas/duvaniste/5e9dd4439a3091338f1a4f3c
https://www.4zida.rs/prodaja/stanovi/nis/oglas/duvaniste/5e6a18d60c7cde136a4a79f2
https://www.4zida.rs/prodaja/stanovi/nis/oglas/crveni-pevac/5ece4edc273171408f47a2b3
https://www.4zida.rs/prodaja/stanovi/nis/oglas/centar/5ef9b42c27317177e56c07a3
https://www.4zida.rs/prodaja/stanovi/nis/oglas/donja-vrezina/5f3bad410c7cde28e03aa3c2
https://www.4zida.rs/prodaja/stanovi/nis/oglas/dom-zdravlja/5f3bab639a30916c3d75fc52
https://www.4zida.rs/prodaja/stanovi/nis/oglas/centar/5f02e4b227317111312f5877
https://www.4zida.rs/prodaja/stanovi/nis/oglas/bulevar-nemanjica/5ee9e64c2731717e5041a767
https://www

https://www.4zida.rs/izdavanje/kuce/nis/oglas/bubanj/5a228854b70c59563744d3d3
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/generala-milojka-lesjanina/5f4424770c7cde761e502f24
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/generala-milojka-lesanina/5f442e7f9a3091742d5d9164
https://www.4zida.rs/izdavanje/stanovi/kragujevac/oglas/centar/5f440f1a7bbec01c45457de3
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/durlan/5f43f7099a3091510b1d85ff
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/centar/5f43dd008fcf7f66f41c2576
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/selicevica/5f2fdde89a30910ebf35a3a3
https://www.4zida.rs/izdavanje/stanovi/kragujevac/oglas/svetozara-markovica/5f43d5a29a30913e231d2a7e
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/dom-zdravlja/5a0c27027ce3af7b702fced1
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/sestre-bakovic/5f439e557b15970e691f2783
https://www.4zida.rs/izdavanje/stanovi/nis/oglas/josifa-pancica-647/5f43a21e0c7cde2bcf46c1ca
https://www.4zid

#### We can see that city data is embeded into the source link. We will extract city info from that link and store it as the correct city info.

In [10]:
def set_city_info_from_source(index):
    """Extracts city infromation from source link and set's up the correct city information."""
    try:
        current_real_estate = real_estate.iloc[index, :] 
        if 'stanovi' in current_real_estate['source']:
            city = current_real_estate['source'].split('stanovi/')[1].split('/')[0]
        elif 'kuce' in current_real_estate['source']:
            city = current_real_estate['source'].split('kuce/')[1].split('/')[0]
        city = city.replace('-', ' ')
        real_estate.at[index, 'location_city'] = city
    except Exception as e:
        pass

In [11]:
print(f'Updating city information on {len(indexes_to_replace)} properties.')
for index in indexes_to_replace:
    set_city_info_from_source(index)

Updating city information on 899 properties.


## 4.2. Properties with "srbija" city value

Properties where serbia was set as the city information. Actual city informatio is here under city district column, so we will just copy city district column value to city value and set city district to be an None value.

In [12]:
# All entries with "Gradske lokacije" city info
indexes_to_replace = real_estate[real_estate['location_city'] == 'srbija'].index
real_estate.iloc[indexes_to_replace, :]

,id,listing_type,property_type,price,location_city,location_city_district,area_property,area_land,construction_type,num_floors_building,apartment_floor,registered,heating_type,num_rooms,num_bathrooms,source
343,478,s,a,36000.0,srbija,sombor opstina,47.0,NaN,2018,4.0,3.0,True,Podno,1.5,1.0,https://www.4zida.rs/prodaja/stanovi/sombor-op...
377,512,s,a,47400.0,srbija,kragujevac,64.0,NaN,None,5.0,3.0,True,Centralno,2.5,1.0,https://www.4zida.rs/prodaja/stanovi/kragujeva...
382,517,s,a,31000.0,srbija,kragujevac,57.0,NaN,None,5.0,5.0,True,Centralno,2.0,1.0,https://www.4zida.rs/prodaja/stanovi/kragujeva...
550,684,s,a,115000.0,srbija,kopaonik opstina,30.0,NaN,None,4.0,1.0,True,Centralno,1.0,1.0,https://www.4zida.rs/prodaja/stanovi/kopaonik-...
633,766,s,a,70000.0,srbija,zlatibor opstina,66.0,NaN,2004,NaN,0.0,True,Etažno,3.0,2.0,https://www.4zida.rs/prodaja/stanovi/zlatibor-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16509,17181,r,a,350.0,srbija,nis,98.0,NaN,2007,5.0,1.0,True,Centralno,3.5,1.0,https://www.4zida.rs/izdavanje/stanovi/nis/ogl...
16909,17597,r,a,90.0,srbija,jagodina opstina,24.0,NaN,None,NaN,1.0,False,Centralno,1.0,1.0,https://www.4zida.rs/izdavanje/stanovi/jagodin...
16953,17642,r,a,300.0,srbija,beograd,38.0,NaN,None,6.0,2.0,False,Struja,1.5,1.0,https://www.4zida.rs/izdavanje/stanovi/beograd...
17739,18477,r,a,200.0,srbija,novi sad,25.0,NaN,1936,NaN,NaN,False,Centralno,1.0,1.0,https://www.4zida.rs/izdavanje/stanovi/novi-sa...


In [13]:
print(f'Updating city information on {len(indexes_to_replace)} properties.')
for index in indexes_to_replace:
    try:
        real_estate.at[index, 'location_city'] = real_estate.iloc[index, :]['location_city_district']
        real_estate.at[index, 'location_city_district'] = None
    except Exception as e:
        pass

Updating city information on 523 properties.


## 4.3. Setting the correct Belgrade's districts

For Belgrade, city information is incorrectly placed. In city column there are actually values for city districts:
- stari grad
- palilula
- vozdovac
- zvezdara
- zemun
- novi beograd
- savski venac
- novi beo
- vracar
- cukarica
- grocka
- rakovica
- obrenovac
- surcin
- barajevo

In [14]:
belgrade_districts = [
    'stari grad',
    'palilula',
    'vozdovac',
    'zvezdara',
    'zemun',
    'novi beograd',
    'savski venac',
    'novi beo',
    'vracar',
    'cukarica',
    'grocka',
    'rakovica',
    'obrenovac',
    'surcin',
    'barajevo',
    'mladenovac'
]

In [15]:
# All entries with wrong Belgrade city info
for belgrade_district in belgrade_districts:
    indexes_to_replace = real_estate[real_estate['location_city'] == belgrade_district].index
    for index in indexes_to_replace:
        real_estate.at[index, 'location_city_district'] = real_estate.iloc[index, :]['location_city']
        real_estate.at[index, 'location_city'] = 'beograd'

## 4.4. Adjusting all properties where in city name says "optsina"

In [16]:
# All entries with "Gradske lokacije" city info
indexes_to_replace = real_estate[real_estate['location_city'].str.contains('opstina')].index
real_estate.iloc[indexes_to_replace, :].head()

,id,listing_type,property_type,price,location_city,location_city_district,area_property,area_land,construction_type,num_floors_building,apartment_floor,registered,heating_type,num_rooms,num_bathrooms,source
55,193,s,a,48500.0,subotica opstina,palic,69.0,NaN,None,NaN,1.0,True,None,2.5,1.0,https://www.4zida.rs/prodaja/stanovi/subotica-...
246,383,s,a,49900.0,subotica opstina,kragujevac,70.0,NaN,None,10.0,9.0,True,Centralno,2.5,1.0,https://www.4zida.rs/prodaja/stanovi/subotica-...
343,478,s,a,36000.0,sombor opstina,None,47.0,NaN,2018,4.0,3.0,True,Podno,1.5,1.0,https://www.4zida.rs/prodaja/stanovi/sombor-op...
347,482,s,a,15000.0,subotica opstina,palic,51.0,NaN,None,NaN,0.0,True,Struja,1.5,1.0,https://www.4zida.rs/prodaja/stanovi/subotica-...
392,527,s,a,37000.0,uzice opstina,kragujevac,57.0,NaN,None,10.0,9.0,True,Centralno,2.0,1.0,https://www.4zida.rs/prodaja/stanovi/uzice-ops...


In [17]:
# Trying to get the right city information from source link
for index in indexes_to_replace:
    print(real_estate.iloc[index,-1])

https://www.4zida.rs/prodaja/stanovi/subotica-opstina/oglas/palic/5f3e56ad4a8b220a123863e3
https://www.4zida.rs/prodaja/stanovi/subotica-opstina/oglas/kertvaros/5f411450872f2d70e052afd3
https://www.4zida.rs/prodaja/stanovi/sombor-opstina/oglas/staparski-put/5c646aa89a30915416332be3
https://www.4zida.rs/prodaja/stanovi/subotica-opstina/oglas/palic/5f40673b9a309115962cc7f0
https://www.4zida.rs/prodaja/stanovi/uzice-opstina/oglas/omladinska/5f3ff045375936599a6cda82
https://www.4zida.rs/prodaja/stanovi/kopaonik-opstina/oglas/kopaonik/5d0a92129a309104f611c209
https://www.4zida.rs/prodaja/stanovi/vrnjacka-banja-opstina/oglas/cajkino-brdo/5f1619119f41cc783d01d6b6
https://www.4zida.rs/prodaja/stanovi/zlatibor-opstina/oglas/narcisa-43/5a2a7ec2b70c5954b7485f72
https://www.4zida.rs/prodaja/stanovi/cacak-opstina/oglas/cacanski-partizanski-odred/5dd13088e466e60b781e0ced
https://www.4zida.rs/prodaja/stanovi/stara-pazova-opstina/oglas/kralja-petra-i-karadjordjevica/5f156da19a30917878008386
https://ww

https://www.4zida.rs/prodaja/kuce/bajina-basta-opstina/oglas/vuka-karadzica/5bfd2dc527317170b573bae2
https://www.4zida.rs/prodaja/kuce/zabalj-opstina/oglas/salas/5f0c1f170c7cde530b3f0fce
https://www.4zida.rs/prodaja/kuce/kraljevo-opstina/oglas/oplanici/5f340c299a3091601758fb55
https://www.4zida.rs/prodaja/kuce/stara-pazova-opstina/oglas/vuka-karadzica/5b718ef5ad85027c827c4752
https://www.4zida.rs/prodaja/kuce/vrnjacka-banja-opstina/oglas/rudjinci/5e6f53eb0c7cde70361a4906
https://www.4zida.rs/prodaja/kuce/vrnjacka-banja-opstina/oglas/borjak/5e6f52b19a3091308f536f66
https://www.4zida.rs/prodaja/kuce/subotica-opstina/oglas/novo-selo/5f34117d1aa7305924073723
https://www.4zida.rs/prodaja/kuce/majdanpek-opstina/oglas/mosna/5da6d97e27317121095db9c2
https://www.4zida.rs/prodaja/kuce/stara-pazova-opstina/oglas/stara-pazova-grad/5b630411b70c5929fe245ce2
https://www.4zida.rs/prodaja/kuce/vrbas-opstina/oglas/sremska/5b55af19b70c597f635c1c02
https://www.4zida.rs/prodaja/kuce/odzaci-opstina/oglas/vo

In [18]:
print(f'Updating location information on {len(indexes_to_replace)} ')
for index in indexes_to_replace:
    real_estate.at[index, 'location_city_district'] = real_estate.iloc[index,:]['location_city']
    real_estate.at[index, 'location_city'] = real_estate.iloc[index,:]['location_city'].replace('opstina', '').strip()

Updating location information on 1378 


## 4.5. Setting the correct Novi Sad's districts

For Novi Sad, city information is incorrectly placed. In city column there are actually values for city districts:
- telep
- novi sad - centar
- veternik
- bulevar oslobodjenja


In [19]:
novi_sad_districts = [
    'telep',
    'novi sad - centar',
    'veternik',
    'bulevar oslobodjenja',
    'petrovaradin'
]

In [20]:
# All entries with wrong Novi Sad's city info
for novi_sad_district in novi_sad_districts:
    indexes_to_replace = real_estate[real_estate['location_city'] == novi_sad_district].index
    for index in indexes_to_replace:
        real_estate.at[index, 'location_city_district'] = real_estate.iloc[index, :]['location_city']
        real_estate.at[index, 'location_city'] = 'novi sad'

## 4.6. Setting the correct city name for "medijana" city district (Nis)

In [21]:
# All entries with "Gradske lokacije" city info
indexes_to_replace = real_estate[real_estate['location_city'] == 'medijana'].index
real_estate.iloc[indexes_to_replace, :].head()

,id,listing_type,property_type,price,location_city,location_city_district,area_property,area_land,construction_type,num_floors_building,apartment_floor,registered,heating_type,num_rooms,num_bathrooms,source
126,264,s,a,77000.0,medijana,bulevar nemanjica,75.0,NaN,None,NaN,2.0,True,Struja,3.0,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
132,270,s,a,54400.0,medijana,gradske lokacije,64.0,NaN,None,3.0,2.0,True,Gas,3.0,1.0,https://www.4zida.rs/prodaja/stanovi/subotica-...
136,274,s,a,56500.0,medijana,bulevar nemanjica,62.0,NaN,None,7.0,6.0,True,Centralno,2.0,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
230,367,s,a,62500.0,medijana,bulevar nemanjica,84.0,NaN,None,NaN,8.0,False,Centralno,3.5,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...
238,375,s,a,35000.0,medijana,bulevar dr zorana djindjica,42.0,NaN,2003,6.0,5.0,True,TA peć,1.5,1.0,https://www.4zida.rs/prodaja/stanovi/nis/oglas...


In [22]:
print(f'Updating city information on {len(indexes_to_replace)} properties.')
for index in indexes_to_replace:
    try:
        real_estate.at[index, 'location_city_district'] = real_estate.iloc[index, :]['location_city']
        real_estate.at[index, 'location_city'] = 'nis'
    except Exception as e:
        pass

Updating city information on 77 properties.


## 4.7. Removing entries that contain "okolne lokacije" value for city district

In [23]:
indexes_to_remove = real_estate[real_estate['location_city'] == 'okolne lokacije'].index

In [24]:
print(f'There are {len(indexes_to_remove)} real estate entries entries to be removed.')

There are 18 real estate entries entries to be removed.


In [25]:
real_estate = real_estate.drop(indexes_to_remove)

# 5. Saving cleaned dataset

In [26]:
len(real_estate)

21586

## 5.1. Exporting to .csv file

In [27]:
# Saving results to the csv file
real_estate.to_csv('../database/backup/properties_cleaned_data.csv',index=False)

## 5.2. Inserting into database

Saving cleaned results into new table, table that containes final, cleaned data, ready for analysis.

In [28]:
# Creating new PostgreSQL engine 
from sqlalchemy import create_engine

postgres_engine_parameters = "postgresql://{}:{}@{}:{}/{}".format(
    db_params['user'],
    db_params['password'],
    db_params['host'],
    db_params['port'],
    db_params['database'],
)

postgres_engine = create_engine(postgres_engine_parameters)

In [29]:
# Inserting cleaned data into "real_estate" table via sqlalchemy engine 
real_estate.to_sql('real_estate', postgres_engine, index=False)

# 6. Closing database connection

In [30]:
# Closing connection
if conn:
    conn.close()